# AWS IoT Core - Over MQTT
 - Applicable only for ESP32
 - Resource link: https://aws.amazon.com/blogs/iot/using-micropython-to-get-started-with-aws-iot-core/

## Policy document

In [ ]:
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": "iot:Connect",
      "Resource": "arn:aws:iot:ap-south-1:291302058818:client/espClient1"
    },
    {
      "Effect": "Allow",
      "Action": "iot:Publish",
      "Resource": "arn:aws:iot:ap-south-1:291302058818:topic/$aws/things/espThing1/shadow/update"
    },
    {
      "Effect": "Allow",
      "Action": "iot:Subscribe",
      "Resource": "arn:aws:iot:ap-south-1:291302058818:topicfilter/$aws/things/espThing1/shadow/update/delta"
    },
    {
      "Effect": "Allow",
      "Action": "iot:Receive",
      "Resource": "arn:aws:iot:ap-south-1:291302058818:topic/$aws/things/espThing1/shadow/update/delta"
    }
  ]
}

## Code
reference link: https://github.com/aws-samples/aws-iot-core-getting-started-micropython/blob/main/main.py

In [ ]:
import os
import time
import ujson
import machine
import network
from umqtt.simple import MQTTClient

#Enter your wifi SSID and password below.
wifi_ssid = "SSID"
wifi_password = "Password"

#Enter your AWS IoT endpoint. You can find it in the Settings page of
#your AWS IoT Core console.
#https://docs.aws.amazon.com/iot/latest/developerguide/iot-connect-devices.html
aws_endpoint = b'a3b4cc29u6m97v-ats.iot.ap-south-1.amazonaws.com'

#If you followed the blog, these names are already set.
thing_name = "espThing1"
client_id = "espClient1"
private_key = "private.pem.key"
private_cert = "cert.pem.crt"

#Read the files used to authenticate to AWS IoT Core
with open(private_key, 'r') as f:
    key = f.read()
with open(private_cert, 'r') as f:
    cert = f.read()

#These are the topics we will subscribe to. We will publish updates to /update.
#We will subscribe to the /update/delta topic to look for changes in the device shadow.
topic_pub = "$aws/things/" + thing_name + "/shadow/update"
topic_sub = "$aws/things/" + thing_name + "/shadow/update/delta"
ssl_params = {"key":key, "cert":cert, "server_side":False}

#Define pins for LED and light sensor. In this example we are using a FeatherS2.
#The sensor and LED are built into the board, and no external connections are required.
light_sensor = machine.ADC(machine.Pin(4))
#light_sensor.atten(machine.ADC.ATTN_11DB)
led = machine.Pin(2, machine.Pin.OUT)
info = os.uname()

#Connect to the wireless network
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
if not wlan.isconnected():
    print('Connecting to network...')
    wlan.connect(wifi_ssid, wifi_password)
    while not wlan.isconnected():
        pass

    print('Connection successful')
    print('Network config:', wlan.ifconfig())

def mqtt_connect(client=client_id, endpoint=aws_endpoint, sslp=ssl_params):
    mqtt = MQTTClient(client_id=client, server=endpoint, port=8883, keepalive=4000, ssl=True, ssl_params=sslp)
    print("Connecting to AWS IoT...")
    mqtt.connect()
    print("Done")
    return mqtt

def mqtt_publish(client, topic=topic_pub, message=''):
    print("Publishing message...")
    client.publish(topic, message)
    print(message)

def mqtt_subscribe(topic, msg):
    print("Message received...")
    message = ujson.loads(msg)
    print(topic, message)
    if message['state']['led']:
        led_state(message)
    print("Done")

def led_state(message):
    led.value(message['state']['led']['onboard'])

#We use our helper function to connect to AWS IoT Core.
#The callback function mqtt_subscribe is what will be called if we
#get a new message on topic_sub.

light =
try:
    mqtt = mqtt_connect()
    mqtt.set_callback(mqtt_subscribe)
    mqtt.subscribe(topic_sub)
except:
    print("Unable to connect to MQTT.")


while True:
#Check for messages.
    try:
        mqtt.check_msg()
    except:
        print("Unable to check for messages.")

    mesg = ujson.dumps({
        "state":{
            "reported": {
                "device": {
                    "client": client_id,
                    "uptime": time.ticks_ms(),
                    "hardware": info[0],
                    "firmware": info[2]
                },
                "sensors": {
                    "light": light,
                    "sound":
                },
                "led": {
                    "onboard": led.value()
                }
            }
        }
    })

#Using the message above, the device shadow is updated.
    try:
        mqtt_publish(client=mqtt, message=mesg)
    except:
        print("Unable to publish message.")

#Wait for 10 seconds before checking for messages and publishing a new update.
    print("Sleep for 10 seconds")
    time.sleep(10)

# Thingspeak - using REST API

In [ ]:
import network
import time

print("Connecting to WiFi", end="")
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect('Wokwi-GUEST', '')
while not sta_if.isconnected():
  print(".", end="")
  time.sleep(0.1)
print(" Connected!")
print('network config:', sta_if.ifconfig())

import urequests as requests

import json
import random

HTTP_HEADERS = {'Content-Type': 'application/json'}
THINGSPEAK_WRITE_API_KEY = "DDI8U1PVBN94GZO1"
sensor = random.randint(10, 50)

data = {'field1':sensor}

while True:
  request = requests.post(
            'http://api.thingspeak.com/update?api_key=' +
            THINGSPEAK_WRITE_API_KEY,
            json = data,
            headers = HTTP_HEADERS )
  request.close()
  time.sleep(15)

# Receive data on computer using MQTT

In [ ]:
! pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=2b1110a2ffb038f79ce7632eb54a7ce025d810d8b34afebbf5703917ae1e9d3d
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


In [ ]:

import paho.mqtt.client as mqtt

# Replace with your MQTT broker details
mqtt_broker = "broker.mqttdashboard.com"
mqtt_port = 1883
mqtt_topic = "atul_test_1234"

# Callback function to handle incoming MQTT messages
def on_message(client, userdata, message):
    # Get the data from the MQTT message payload
    data = message.payload.decode()
    print(data)



# Create an MQTT client and connect to the broker
client = mqtt.Client()
client.connect(mqtt_broker, mqtt_port)

# Subscribe to the MQTT topic
client.subscribe(mqtt_topic)

# Set the callback function for incoming MQTT messages
client.on_message = on_message

# Start the MQTT loop
client.loop_forever()